In [1]:
from gurobipy import *

In [2]:
def formulation():
    # declare model
    model_jeans = Model(name='jeans_production')
    
    # decision variables x_i, integers
    # the number of type i workers to hire, i = 0 ~ 3 (Python index starts from 0)
    # 4 types in total:
    # type 0 works Q1,Q2,Q3
    # type 1 works Q2,Q3,Q4
    # type 2 works Q3,Q4,Q1
    # type 3 works Q4,Q1,Q2
    x = model_jeans.addVars(4, lb=0, vtype=GRB.INTEGER, name='worker')
    
    # decision variables y_i
    # total production in quarter i+1, i = 0 ~ 3
    y = model_jeans.addVars(4, lb=0, vtype=GRB.INTEGER, name='production')
    
    # derived variables z_i
    # total inventory at the end of quarter i+1, i = 0 ~ 3
    z = model_jeans.addVars(4, lb=0, vtype=GRB.INTEGER, name='inventory')
    
    # objective function
    # minimize total cost: labor cost + holding cost
    # = 600*3*(x0+x1+x2+x3) + 10*(z0+z1+z2)
    # (if -20*z3 is included, the problem is unbounded because z3 can be infinitely large)
    model_jeans.setObjective(600 * 3 * quicksum(x[i] for i in range(4))
                             + 10 * quicksum(z[i] for i in range(3)),
                             GRB.MINIMIZE)
    
    # production constraints, up to 600 pairs of jeans per worker
    # (quarter_schedule[0] = [1,0,1,1] means x0,x2,x3 are working in Q1, etc.)
    quarter_schedule = [[1,0,1,1], [1,1,0,1], [1,1,1,0], [0,1,1,1]]
    model_jeans.addConstrs(y[i] <= 600 * quicksum(quarter_schedule[i][j] * x[j] for j in range(4))
                                   for i in range(4))
    
    # parameters
    # demand d_i of quarter i+1, i = 0 ~ 3
    d = [4000, 7000, 8000, 3000]
    
    # inventory constraints
    model_jeans.addConstr(z[0] == y[0] - d[0])                                # Q1
    model_jeans.addConstrs(z[i] == z[i-1] + y[i] - d[i] for i in range(1,4))  # Q2 ~ Q4
    
    return model_jeans

In [3]:
def check_solve(model):
    model.printStats()
    print()
    model.optimize()
    model.printQuality()
    return model.objVal

In [4]:
model_jeans = formulation()

Using license file C:\Users\yuwen\gurobi.lic
Academic license - for non-commercial use only


In [5]:
total_cost = check_solve(model_jeans)


Statistics for model jeans_production :
  Linear constraint matrix    : 0 Constrs, 0 Vars, 0 NZs
  Matrix coefficient range    : [ 0, 0 ]
  Objective coefficient range : [ 0, 0 ]
  Variable bound range        : [ 0, 0 ]
  RHS coefficient range       : [ 0, 0 ]

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 8 rows, 12 columns and 27 nonzeros
Model fingerprint: 0x985b3b08
Variable types: 0 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+01, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+03, 8e+03]
Found heuristic solution: objective 46800.000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 22 nonzeros
Variable types: 0 continuous, 9 integer (0 binary)

Root relaxation: objective 2.400000e+04, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Inc

# Print value for decision variables

In [6]:
model_jeans.printAttr('X')


    Variable            X 
-------------------------
   worker[0]            8 
   worker[1]            6 
production[0]         4000 
production[1]         7000 
production[2]         8000 
production[3]         3600 
inventory[3]          600 


In [8]:
print('Total cost: {}'.format(total_cost))

print()
print('Number of workers to hire:')
x = [v.x for v in model_jeans.getVars() if v.varName.startswith('worker')]
for i in range(len(x)):
    print('Type {}: {}'.format(i, x[i]))

print()
print('Total production of:')
y = [v.x for v in model_jeans.getVars() if v.varName.startswith('production')]
for i in range(len(y)):
    print('Quarter {}: {}'.format(i+1, y[i]))
    
print()
print('Inventory at the end of')
z = [v.x for v in model_jeans.getVars() if v.varName.startswith('inventory')]
for i in range(len(y)):
    print('Quarter {}: {}'.format(i+1, z[i]))

Total cost: 25200.0

Number of workers to hire:
Type 0: 8.0
Type 1: 6.0
Type 2: -0.0
Type 3: -0.0

Total production of each quarter:
Quarter 1: 4000.0
Quarter 2: 7000.0
Quarter 3: 8000.0
Quarter 4: 3600.0

Inventory at the end of
Quarter 1: 0.0
Quarter 2: -0.0
Quarter 3: -0.0
Quarter 4: 600.0
